***AVISO**: Esse Notebook foi feito com base na estrutura do Databricks Free Edition, que utiliza catálogos.*

# **ETAPA 0 - CRIAÇÃO DO DATA LAKEHOUSE E SUAS CAMADAS**

---
---

<br>

Após as etapas de _Levantamento de Requisitos_, _Identificação da Base de Dados_, _Geração do Modelo Conceitual_ e _Verificação pós-mesclagem do Modelo Conceitual com a Base de Dados_. Agora começa a etapa prática do projeto, iniciando pela criação do ambiente onde todos os dados ficarão organizados no Delta Lakehouse. 

Será utilizada a estrutura:
- **Landing Zone** -> Dados totalmente brutos, em seus formatos originais
- **Bronze** -> Dados ainda brutos, porém já com formato padrão
- **Silver** -> Dados limpos e filtrados
- **Gold** -> Dados agrupados, prontos para criação de modelo semânticos e utilização externa.

Em resumo, as camadas Landing e Bronze lembram de tudo, a camada Silver entende o passado e a Gold responde o presente.

*`Complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br><br>

---
---

### Parte 1 - **Importação das Bibliotecas Necessárias**

> _Nessa etapa não há necessidade de importação de bibliotecas_



### Parte 2 - **Criação do Data Lakehouse**

`Complete a variável:`<br>
- `nome_datalakehouse --> Nome do Data Lakehouse`


In [0]:
nome_datalakehouse = "dataexperts"

Aqui será criado uma **variável** para ajudar a **validar** se o Data Lakehouse que estamos criando foi realmente criado **nesta execução**:

In [0]:
datalakehouse_foi_criado_agora = True

Aqui será o código para buscar todos os Data Lakehouses existentes e armazenar o resultado dessa consulta em um Data Frame, assim verificando:
  - Se já existir, não acontecerá mais no código para não prejudicar outros projetos que possam já existir.
  - Se ainda não existir, cria toda a estrutura esperada, tanto o Data Lakehouse quanto as suas respectivas camadas.

In [0]:
dataslakeshouses_existentes = spark.sql("SHOW CATALOGS")

datalakehouse_foi_criado_agora = not (
    dataslakeshouses_existentes
    .filter(dataslakeshouses_existentes.catalog == nome_datalakehouse)
    .count() > 0
)

Aqui será o código responsável por **criar** o **Data Lakehouse (catálogo)** junto do nome personalizado anteriormente e já colocá-lo em uso:

In [0]:
if datalakehouse_foi_criado_agora:
  spark.sql(f"CREATE CATALOG {nome_datalakehouse}")
  spark.sql(f"USE CATALOG {nome_datalakehouse}")

### Parte 3 - **Criação das Partições do Data LakeHouse** 

`Complete a variável:`<br>
- `nomes_camadas --> Nome de todas as camadas que deseja na estrutura`

In [0]:
nomes_camadas = [
    "landing_zone", 
    "bronze", 
    "silver", 
    "gold"
]

Aqui será o código para criar as camadas separadamente:

In [0]:
if datalakehouse_foi_criado_agora:
  for camada in nomes_camadas:
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {camada}")  


<br>

---
---

### **Resultados**

Código simples para mostrar se deu certo ou não essa etapa:

In [0]:
# %skip
if datalakehouse_foi_criado_agora:
    print(f"[INFO] O Data LakeHouse '{nome_datalakehouse}' e suas respectivas camadas foram criadas com sucesso!")
else:
    print(f"[INFO] O Data LakeHouse '{nome_datalakehouse}' já existe, escolha outro nome!")


[INFO] O Data LakeHouse 'dataexperts' e suas respectivas camadas foram criadas com sucesso!


---
---

<br><br><br><br><br>

# **ETAPA 1 - IMPORTAÇÃO DAS BASES DE DADOS**

### Parte 1 - **Importação das Bibliotecas Necessárias**

In [0]:
import urllib
import datetime

### Parte 2 - **Importação dos Arquivos para o Data Lakehouse**

`Insira nas variáveis:` <br>
- `nome_datalakehouse --> nome do Data Lakehouse destino`
- `nome_camada --> o nome da camada destino dos dados totalmente brutos`
- `nome_volume_atual --> nome do volume onde ficaram os dados atualizados`
- `nome_volume_historico --> nome do volume onde ficaram o histórico dos dados`

In [0]:
nome_datalakehouse = "dataexperts"
nome_camada = "landing_zone"
nome_volume_atual = "vendas_atual"
nome_volume_historico = "vendas_historico"

O código a seguir cria os volumes se eles ainda não existirem:

In [0]:
spark.sql(f"""
CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada}.{nome_volume_atual}
""")

spark.sql(f"""
CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada}.{nome_volume_historico}
""")

DataFrame[]

`Insira nos vetores todos os nomes dos arquivos que deseja importar e a origem deles, seguindo a mesma ordem:`

In [0]:
arquivos = [
    "categoria_produto.csv",
    "cliente.csv",
    "data.csv",
    "localidade.csv",
    "produto.csv",
    "vendas_part1.csv",
    "vendas_part2.csv",
    "vendas_part3.csv",
    "vendas_part4.csv"
]

origem_arquivos = [
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
]

O código a seguir faz o download do arquivo seguindo a seguinte lógica:
- Se o arquivo não existir no sistema ainda, joga ele para a pasta dos arquivos atuais e registra a data no final do seu nome
- Se o arquivo já existir, então o mais novo vai para a pasta atual e o anterior igual vai para pasta de histórico, separada granularmente pela data para facilitar consultas futuras.

In [0]:
destino_atual = f"/Volumes/{nome_datalakehouse}/{nome_camada}/{nome_volume_atual}/"
destino_historico = f"/Volumes/{nome_datalakehouse}/{nome_camada}/{nome_volume_historico}/"

horario_brasil = datetime.timezone(datetime.timedelta(hours=-3))

for arquivo, origem in zip(arquivos, origem_arquivos):
    nome_arquivo = arquivo.replace(".csv", "")
    
    try:
        arquivos_existentes = dbutils.fs.ls(destino_atual)
        
        for arquivo_existente in arquivos_existentes:
            if arquivo_existente.name.startswith(f"{nome_arquivo}_") and arquivo_existente.name.endswith(".csv"):
                
                nome_sem_extensao = arquivo_existente.name.replace(".csv", "")
                partes = nome_sem_extensao.split("_")
                data_arquivo = partes[-2] 
                
                ano_arquivo = data_arquivo[0:4]
                mes_arquivo = data_arquivo[4:6]
                dia_arquivo = data_arquivo[6:8]
                
                pasta_historico_real = f"{destino_historico}{nome_arquivo}/{ano_arquivo}/{mes_arquivo}/{dia_arquivo}/"
                
                dbutils.fs.mkdirs(pasta_historico_real)
                
                caminho_historico = f"{pasta_historico_real}{arquivo_existente.name}"
                dbutils.fs.mv(arquivo_existente.path, caminho_historico)
                
                print(f"[INFO] Histórico criado e arquivo movido: {caminho_historico}")
                
    except Exception as e:
        pass

    data_hora_agora = datetime.datetime.now(datetime.timezone.utc).astimezone(horario_brasil)
    data_hora_agora_formatada= data_hora_agora.strftime("%Y%m%d_%H%M%S")
    
    nome_arquivo_novo = f"{nome_arquivo}_{data_hora_agora_formatada}.csv"
    caminho_novo = f"{destino_atual}{nome_arquivo_novo}"

    try:
        urllib.request.urlretrieve(origem + arquivo, caminho_novo)
        print(f"[INFO] Novo arquivo {nome_arquivo_novo} baixado com sucesso.")
    except Exception as e:
        print(f"[ERRO] Falha ao baixar {arquivo}: {e}")

[INFO] Novo arquivo categoria_produto_20260206_051843.csv baixado com sucesso.
[INFO] Novo arquivo cliente_20260206_051849.csv baixado com sucesso.
[INFO] Novo arquivo data_20260206_051850.csv baixado com sucesso.
[INFO] Novo arquivo localidade_20260206_051850.csv baixado com sucesso.
[INFO] Novo arquivo produto_20260206_051851.csv baixado com sucesso.
[INFO] Novo arquivo vendas_part1_20260206_051851.csv baixado com sucesso.
[INFO] Novo arquivo vendas_part2_20260206_051852.csv baixado com sucesso.
[INFO] Novo arquivo vendas_part3_20260206_051853.csv baixado com sucesso.
[INFO] Novo arquivo vendas_part4_20260206_051855.csv baixado com sucesso.



<br>

---
---

### **Resultados**

Código simples para mostrar se deu certo ou não essa etapa:

- Mostrar todos os arquivos mais recentes salvos:

In [0]:
# %skip
display(dbutils.fs.ls(destino_atual))

path,name,size,modificationTime
dbfs:/Volumes/dataexperts/landing_zone/vendas_atual/categoria_produto_20260206_051843.csv,categoria_produto_20260206_051843.csv,184,1770365930000
dbfs:/Volumes/dataexperts/landing_zone/vendas_atual/cliente_20260206_051849.csv,cliente_20260206_051849.csv,31088,1770365931000
dbfs:/Volumes/dataexperts/landing_zone/vendas_atual/data_20260206_051850.csv,data_20260206_051850.csv,57386,1770365931000
dbfs:/Volumes/dataexperts/landing_zone/vendas_atual/localidade_20260206_051850.csv,localidade_20260206_051850.csv,1532,1770365931000
dbfs:/Volumes/dataexperts/landing_zone/vendas_atual/produto_20260206_051851.csv,produto_20260206_051851.csv,6144,1770365932000
dbfs:/Volumes/dataexperts/landing_zone/vendas_atual/vendas_part1_20260206_051851.csv,vendas_part1_20260206_051851.csv,21144859,1770365933000
dbfs:/Volumes/dataexperts/landing_zone/vendas_atual/vendas_part2_20260206_051852.csv,vendas_part2_20260206_051852.csv,21255529,1770365933000
dbfs:/Volumes/dataexperts/landing_zone/vendas_atual/vendas_part3_20260206_051853.csv,vendas_part3_20260206_051853.csv,21755057,1770365935000
dbfs:/Volumes/dataexperts/landing_zone/vendas_atual/vendas_part4_20260206_051855.csv,vendas_part4_20260206_051855.csv,21757725,1770365936000


- Mostrar o histórico de todas as versões dos arquivos salvos:

In [0]:
# %skip
display(dbutils.fs.ls(destino_historico))

[]

- Mostrar um arquivo como exemplo:

In [0]:
# %skip
nome_arquivo = arquivos[4].split('.')[0]
arquivos_no_diretorio = dbutils.fs.ls(destino_atual)

arquivo_alvo = None

for arquivo in arquivos_no_diretorio:
    if arquivo.name.startswith(nome_arquivo):
        arquivo_alvo = arquivo.path
        break

if arquivo_alvo:
    dataframe_arquivo_exemplo = spark.read.csv(arquivo_alvo, header=True, inferSchema=True)
    display(dataframe_arquivo_exemplo)
else:
    print("[INFO] Arquivo não encontrado!")

produto_id,preco_lista,categoria_nome
1,36.53,Móveis
2,24.95,Bebidas
3,39.99,Brinquedos
4,67.61,Vestuário
5,23.56,Móveis
6,23.56,Esportes
7,69.93,Alimentos
8,42.97,Eletrônicos
9,20.46,Alimentos
10,37.54,Bebidas


---
---

<br><br><br><br><br>

# **ETAPA 2 - PROCESSAMENTO PARA CAMADA BRONZE**

---
---

<br>

Essa etapa será responsável por mover os dados para a camada bronze, colocando em um formato padrão mas sem mais nenhum refinamento, ainda mantendo os dados brutos.

*`Complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br><br>

---
---

### Parte 1 - **Importação das Bibliotecas Necessárias**

In [0]:
import gc
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, lit, current_timestamp
from pyspark.sql.types import *

### Parte 2 - **Otimizar a Sessão com configurações Personalizadas**

Aqui o será configurado algumas propriedades para que o desempenho da sessão seja mais otimizado 
- Define tamanho fixo de partições para o shuffle para melhorar o paralelismo (usar ***número de partições = número de núclos de CPU * 2 ou 3*** para encontrar melhor cenário possível)
- Define o tamanho máximo de partições para evitar muitos arquivos pequenos
- Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita
- Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados

In [0]:
spark = (
    SparkSession.builder.appName("Load Data Bronze")
    .config("spark.sql.shuffle.partitions", "200")
    .config("spark.sql.files.maxPartitionBytes", "128MB")
    .config("spark.sql.parquet.compression.codec", "snappy")
    .config("spark.sql.adaptive.enabled", "true")
    .getOrCreate()
)

### Parte 3 - **Definindo Origens, Arquivos e Destinos**

`Insira nas variáveis:` <br>
- `nome_camada_origem --> nome da camada de origem dos dados`
- `nome_volume_origem --> nome do volume de origem dos dados dentro da camada` 
- `nome_camada_bronze --> nome da camada de destino dos dados`
- `nome_volume_atual --> nome do volume de destino dos dados mais atualizados dentro da camada`
- `nome_volume_historico --> nome do volume de destino dos dados antigos dentro da camada` 

In [0]:
nome_camada_origem = "landing_zone"
nome_volume_origem = "vendas_atual"

nome_camada_bronze = "bronze"
nome_volume_atual = "vendas_atual" 
nome_volume_historico = "vendas_historico" 

O código a seguir armazena em variáveis os caminhos já prontos de origem e de destino dos dados:

In [0]:
origem_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_origem}/{nome_volume_origem}/"

destino_atual = f"/Volumes/{nome_datalakehouse}/{nome_camada_bronze}/{nome_volume_atual}/"
destino_historico = f"/Volumes/{nome_datalakehouse}/{nome_camada_bronze}/{nome_volume_historico}/" 

O código a seguir cria a estrutura dos volumes de destino, caso eles ainda não existam:

In [0]:
spark.sql(f"CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada_bronze}.{nome_volume_atual}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada_bronze}.{nome_volume_historico}")

DataFrame[]

`Insira no vetor todos os arquivos que deseja fazer esse processamento para camada bronze:` <br>

In [0]:
arquivos_landing_zone = [
    "categoria_produto.csv",
    "cliente.csv",
    "data.csv",
    "localidade.csv",
    "produto.csv",
    "vendas_part1.csv",
    "vendas_part2.csv",
    "vendas_part3.csv",
    "vendas_part4.csv"
]

### Parte 4 - **Leitura dos Dados**

Para que seja possível manipular os dados, antes precisamos ler todos eles já com uma estrutura pré-definida.

`Para cada estrutura, é necessário elaborar manualmente como deve ser interpretada`

O código a seguir cria essas estruturas para cada arquivo diferente:

In [0]:
schemas = {
    "categoria_produto.csv": StructType(
        [
            StructField("categoria_id", LongType()),
            StructField("categoria_nome", StringType()),
        ]
    ),
    "cliente.csv": StructType(
        [
            StructField("cliente_id", LongType()),
            StructField("nome_cliente", StringType()),
            StructField("estado", StringType()),
            StructField("cidade", StringType()),
        ]
    ),
    "data.csv": StructType(
        [
            StructField("data_id", LongType()),
            StructField("data", DateType()),
            StructField("ano", IntegerType()),
            StructField("mes", IntegerType()),
            StructField("dia", IntegerType()),
            StructField("dia_semana", StringType()),
            StructField("final_de_semana", ByteType()),
        ]
    ),
    "localidade.csv": StructType(
        [
            StructField("localidade_id", LongType()),
            StructField("estado", StringType()),
            StructField("cidade", StringType()),
        ]
    ),
    "produto.csv": StructType(
        [
            StructField("produto_id", LongType()),
            StructField("preco_lista", DoubleType()),
            StructField("categoria_nome", StringType()),
        ]
    ),
    "vendas": StructType(
        [
            StructField("venda_id", LongType()),
            StructField("cliente_id", LongType()),
            StructField("produto_id", LongType()),
            StructField("data_id", LongType()),
            StructField("categoria_id", LongType()),
            StructField("localidade_id", LongType()),
            StructField("quantidade", LongType()),
            StructField("preco_lista", DoubleType()),
            StructField("valor_total", DoubleType()),
        ]
    ),
}

O código a seguir faz a leitura dos dados em cada estrutura, adicionando a coluna nome do arquivo de origem e a data que foi feito. armazenando o resultado desse processo em um vetor de datas frames para que seja possível manipular esses dados.

In [0]:
dataframes = {}

try:
    arquivos_existentes = dbutils.fs.ls(origem_dados)
except Exception as e:
    print(f"[ERRO] Falha ao acessar volume: {e}")
    arquivos_existentes = []

for arquivo_origem in arquivos_landing_zone:
    try:
        nome_arquivo = arquivo_origem.replace(".csv", "")
        arquivo_encontrado = None

        for arquivo_existente in arquivos_existentes:
            if arquivo_existente.name.startswith(f"{nome_arquivo}_") and arquivo_existente.name.endswith(".csv"):
                arquivo_encontrado = arquivo_existente
                break

        if not arquivo_encontrado:
            print(f"[INFO] Arquivo para '{nome_arquivo}' não encontrado na Landing.")
            continue

        schema = schemas["vendas"] if nome_arquivo.startswith("vendas") else schemas[arquivo_origem]

        dataframe = (
            spark.read.option("header", "true")
            .schema(schema)
            .csv(arquivo_encontrado.path)
            .withColumn("_source_file", lit(arquivo_encontrado.name))
            .withColumn("_ingestion_date", current_timestamp())
        )

        dataframes[nome_arquivo] = dataframe
        print(f"[INFO] Leitura de {arquivo_encontrado.name} realizada.")

    except Exception as e:
        print(f"[ERRO] Falha ao processar {arquivo_origem}: {e}")

[INFO] Leitura de categoria_produto_20260206_051843.csv realizada.
[INFO] Leitura de cliente_20260206_051849.csv realizada.
[INFO] Leitura de data_20260206_051850.csv realizada.
[INFO] Leitura de localidade_20260206_051850.csv realizada.
[INFO] Leitura de produto_20260206_051851.csv realizada.
[INFO] Leitura de vendas_part1_20260206_051851.csv realizada.
[INFO] Leitura de vendas_part2_20260206_051852.csv realizada.
[INFO] Leitura de vendas_part3_20260206_051853.csv realizada.
[INFO] Leitura de vendas_part4_20260206_051855.csv realizada.


### Parte 5 - **Salvando os Dados no Formato e Local Corretos**

Por fim, precisamos criar o parquet com os dados que importamos e com a estrutura que construimos anteriormente e mover para camada bronze.

O código a seguir faz essa compressão e envio para o caminho certo:

In [0]:
data_hora_agora = datetime.datetime.now(datetime.timezone.utc).astimezone(horario_brasil)
ano = data_hora_agora.strftime("%Y")
mes = data_hora_agora.strftime("%m")
dia = data_hora_agora.strftime("%d")
data_hora_agora_formatada = data_hora_agora.strftime("%Y%m%d_%H%M%S")

def salvar_bronze(dataframe_pronto, nome_tabela):
    caminho_atual = f"{destino_atual}/{nome_tabela}"
    
    pasta_historico = f"{destino_historico}/{nome_tabela}/{ano}/{mes}/{dia}/"
    caminho_historico = f"{pasta_historico}/{nome_tabela}_{data_hora_agora_formatada}"

    try:
        dbutils.fs.ls(caminho_atual)
        dbutils.fs.mkdirs(pasta_historico) 
        dbutils.fs.mv(caminho_atual, caminho_historico, recurse=True)
        print(f"[INFO] Tabela anterior de {nome_tabela} arquivada em: {pasta_historico}")
    except:
        pass

    dataframe_pronto.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(caminho_atual)
    print(f"[INFO] {nome_tabela} atualizada com sucesso na Bronze Atual.")

dimensoes = {
    "categoria_produto": "bronze_dim_categoria_produto",
    "cliente": "bronze_dim_cliente",
    "data": "bronze_dim_data",
    "localidade": "bronze_dim_localidade",
    "produto": "bronze_dim_produto"
}

for nome_arquivo_bruto, nome_arquivo_formatado in dimensoes.items():
    if nome_arquivo_bruto in dataframes:
        salvar_bronze(dataframes[nome_arquivo_bruto], nome_arquivo_formatado)

vendas_dataframes = [dataframe_vendas for nome, dataframe_vendas in dataframes.items() if nome.startswith("vendas_part")]

if vendas_dataframes:
    dataframe_vendas_pronto = vendas_dataframes[0]
    for dataframes_outras_partes in vendas_dataframes[1:]:
        dataframe_vendas_pronto = dataframe_vendas_pronto.unionAll(dataframes_outras_partes)
    
    salvar_bronze(dataframe_vendas_pronto, "bronze_fato_vendas")

[INFO] bronze_dim_categoria_produto atualizada com sucesso na Bronze Atual.
[INFO] bronze_dim_cliente atualizada com sucesso na Bronze Atual.
[INFO] bronze_dim_data atualizada com sucesso na Bronze Atual.
[INFO] bronze_dim_localidade atualizada com sucesso na Bronze Atual.
[INFO] bronze_dim_produto atualizada com sucesso na Bronze Atual.
[INFO] bronze_fato_vendas atualizada com sucesso na Bronze Atual.


> O modo de escrita define como os dados serão gravados no destino:
- ***overwrite***: remove os dados existentes no caminho/partições e grava tudo novamente
- ***append***: adiciona novos dados às partições existentes, sem apagar o que já existe 

> Em pipelines produtivos, 'append' é o mais comum para cargas incrementais. 'overwrite' costuma ser usado apenas em reprocessamentos completos ou ambientes de teste

### Parte 6 - **Limpeza de Cache e Outros**

O código a seguir libera memória de objetos não mais utilizados:

In [0]:
for dataframe in dataframes.values():
    del dataframe
gc.collect()

40246


<br>

---
---

### **Resultados**

Código simples para mostrar se deu certo ou não essa etapa:

In [0]:
# %skip
dataframe_bronze = spark.read.format("delta").load(destino_atual + "/bronze_dim_categoria_produto")

display(dataframe_bronze)

categoria_id,categoria_nome,_source_file,_ingestion_date
1,Eletrônicos,categoria_produto_20260206_051843.csv,2026-02-06T08:19:08.987Z
2,Eletrodomésticos,categoria_produto_20260206_051843.csv,2026-02-06T08:19:08.987Z
3,Alimentos,categoria_produto_20260206_051843.csv,2026-02-06T08:19:08.987Z
4,Bebidas,categoria_produto_20260206_051843.csv,2026-02-06T08:19:08.987Z
5,Vestuário,categoria_produto_20260206_051843.csv,2026-02-06T08:19:08.987Z
6,Calçados,categoria_produto_20260206_051843.csv,2026-02-06T08:19:08.987Z
7,Cosméticos,categoria_produto_20260206_051843.csv,2026-02-06T08:19:08.987Z
8,Esportes,categoria_produto_20260206_051843.csv,2026-02-06T08:19:08.987Z
9,Móveis,categoria_produto_20260206_051843.csv,2026-02-06T08:19:08.987Z
10,Brinquedos,categoria_produto_20260206_051843.csv,2026-02-06T08:19:08.987Z
